In [4]:
#pip install sparknlp

     |████████████████████████████████| 142 kB 3.3 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
#pip install nltk

     |████████████████████████████████| 1.5 MB 5.6 MB/s eta 0:00:01
     |████████████████████████████████| 764 kB 7.0 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import udf, col, lower, regexp_replace,countDistinct
from pyspark.ml.feature import Tokenizer, StopWordsRemover, NGram, HashingTF, IDF
from pyspark.ml import Pipeline
from pyspark.sql.types import StructType,StructField
import pandas as pd
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

#from nltk.stem.snowball import SnowballStemmer

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
spark = SparkSession.builder.getOrCreate()

# Chargement des données

In [3]:


#telechargement des données de trainning
train_review = "/home/jovyan/work/data/dataset_booking_prepared2.csv"

# lecture d'un fichier de manière la plus brute
schema = StructType([
    StructField('c0', IntegerType(),True),
    StructField('hotel_name', StringType(),True),
    StructField('lat', FloatType(),True),
    StructField('long', FloatType(),True),
    StructField('average_score', FloatType(),True),
    StructField('review', StringType(),True),
    StructField('polarity', IntegerType(),True),
    StructField('word_counts', IntegerType(),True),
    StructField('tags', StringType(),True),
])

df = spark.read.format('csv').options(header=True).options(delimiter= ";").schema(schema).load(train_review )
df.dtypes

[('c0', 'int'),
 ('hotel_name', 'string'),
 ('lat', 'float'),
 ('long', 'float'),
 ('average_score', 'float'),
 ('review', 'string'),
 ('polarity', 'int'),
 ('word_counts', 'int'),
 ('tags', 'string')]

In [4]:
df= df.dropna(how="any")
df.filter(df.polarity.isNull()).count()

0

In [5]:
df.count()

746400

In [8]:
df.select('polarity').where(df.polarity==0).count()

337284

In [9]:
df.select('polarity').where(df.polarity==1).count()

409116

# MultiModel comparaison 

### Pipeline 

In [138]:
# Tokenize text
tokenizer = Tokenizer(inputCol='review', outputCol='words_token')

#StopWord
stopwordList = nltk.corpus.stopwords.words('english')
remover = StopWordsRemover(inputCol='words_token', outputCol='words_clean', stopWords=stopwordList )

#Ngram
bigram = NGram(n=2, inputCol="words_clean", outputCol="bigrams")

#HashingTF
hashingTF2 = HashingTF(inputCol="bigrams", outputCol="rawFeatures", numFeatures=10000)

#IDF
idf = IDF(inputCol="rawFeatures", outputCol="features")

In [139]:
from pyspark.ml import Pipeline

pipeline2 = Pipeline() \
     .setStages([
           tokenizer,
           remover,
           bigram,
           hashingTF2,
           idf
     ])

In [140]:
# entrainement du pipeline 
pipeline2 = pipeline2.fit(df)

# Enregistrement du pipeline
#pipeline.write().overwrite().save("myPipeline")

In [141]:
df_prepared2= pipeline2.transform(df)

### Split train-Test

In [142]:
split2= df_prepared2.randomSplit([0.7,0.3],100)

train2= split2[0]
#train_dt.count()

test2=split2[1]
#test_dt.count()

## Decision TreeClassifier

In [143]:
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'polarity', maxDepth = 3)
# Entrainement du Modele
dtModel = dt.fit(train2)
# prediction sur données Test
dtPreds = dtModel.transform(test2)
my_eval_dt = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='polarity', metricName='areaUnderROC')
eval_dt10K= my_eval_dt.evaluate(dtPreds)
eval_dt10K

0.5370055050609991

## RandomForest Classifier

In [144]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'polarity')

# Entrainement du Modele
rfModel = rf.fit(train2)

# prediction sur données Test
rfPreds = rfModel.transform(test2)

# Evaluation du modèle
my_eval_rf = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='polarity', metricName='areaUnderROC')
eval_rf10K= my_eval_rf.evaluate(rfPreds)
eval_rf10K

0.5531773611302762

## Gradient-Boosted Tree Classifier

In [145]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(featuresCol = 'features', labelCol = 'polarity',maxIter=10)

# Entrainement du Modele
gbtModel = gbt.fit(train2)

# prediction sur données Test
gbtPreds = gbtModel.transform(test2)

# Evaluation du modèle
my_eval_gbt = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='polarity', metricName='areaUnderROC')
eval_gbt10K= my_eval_gbt.evaluate(gbtPreds) 
eval_gbt10K

0.7503369181607551

## Regression Logistique

In [146]:
from pyspark.ml.classification import LogisticRegression

lr2 = LogisticRegression(featuresCol = 'features', labelCol = 'polarity', maxIter=10)

# Entrainement du Modele
lr2Model = lr2.fit(train2)

# prediction sur données Test
lr2Preds= lr2Model.transform(test2)

# Evaluation du modèle
my_eval_lr2 = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='polarity', metricName='areaUnderROC')
eval10K_lr2= my_eval_lr2.evaluate(lr2Preds)
eval10K_lr2

0.8495708125267121

In [ ]:
#df100 = pd.DataFrame({'DT': eval_dt, 'RF': eval_rf,'gbt': eval_gbt, 'lr': eval_lr2},
                      #index = ['num=100'])
df100

In [133]:
#df10 = pd.DataFrame({'DT': eval_dt10, 'RF': eval_rf10,'gbt': eval_gbt10, 'lr': eval10_lr2},
                      #index = ['num=10'])
#df10

,DT,RF,gbt,lr
num=10,0.571381,0.56783,0.57264,0.516648


In [136]:
#df1000 = pd.DataFrame({'DT': eval_dt1000, 'RF': eval_rf10000,'gbt': eval_gbt10000, 'lr': eval10000_lr2},
                      #index = ['num=1000'])
#df1000

,DT,RF,gbt,lr
num=1000,0.538062,0.548387,0.703538,0.765896


In [149]:
#df10K = pd.DataFrame({'DT': eval_dt10K, 'RF': eval_rf10K,'gbt': eval_gbt10K, 'lr': eval10K_lr2},
                      #index = ['num=10000'])
#df10K

,DT,RF,gbt,lr
num=10000,0.537006,0.553177,0.750337,0.849571


## Comparatif de la performance des modèles

In [151]:
df_evaluation = pd.concat([df10,df100,df1000,df10K], ignore_index=False)
df_evaluation

,DT,RF,gbt,lr
num=10,0.571381,0.567830,0.572640,0.516648
num=100,0.565484,0.560384,0.616621,0.634291
num=1000,0.538062,0.548387,0.703538,0.765896
num=10000,0.537006,0.553177,0.750337,0.849571


# Optimisation LogisticRegression Modele

## Pipeline

In [5]:
# Tokenize text
tokenizer = Tokenizer(inputCol='review', outputCol='words_token')

#StopWord
stopwordList = nltk.corpus.stopwords.words('english')
remover = StopWordsRemover(inputCol='words_token', outputCol='words_clean', stopWords=stopwordList )

#Ngram
bigram = NGram(n=2, inputCol="words_clean", outputCol="bigrams")

#HashingTF
hashingTF = HashingTF(inputCol="bigrams", outputCol="rawFeatures", numFeatures=1000000)

#IDF
idf = IDF(inputCol="rawFeatures", outputCol="features")



In [6]:
from pyspark.ml import Pipeline

pipeline = Pipeline() \
     .setStages([
           tokenizer,
           remover,
           bigram,
           hashingTF,
           idf
     ])

In [7]:
# entrainement du pipeline 
pipeline = pipeline.fit(df)

# Enregistrement du pipeline
#pipeline.write().overwrite().save("myPipeline")

In [8]:
df_prepared= pipeline.transform(df)

### Split train-Test

In [9]:
split= df_prepared.randomSplit([0.7,0.3],100)

train= split[0]
#train_dt.count()

test=split[1]
#test_dt.count()

## Model LR

In [10]:
# Entrainement du Modèle

from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol = 'features', labelCol = 'polarity', maxIter=10)
lrModel = lr.fit(train)

#print('Coefficient:' + str(lrModel.coefficients))
#print('Intercept:' + str(lrModel.intercept))

### Sauvegarde du modele

In [ ]:
# Save
#lrModel.save("mylrModel")

### Prediction

In [11]:
lrPreds= lrModel.transform(test)

In [12]:
lrPreds.select('hotel_name','polarity','prediction').tail(20)

[Row(hotel_name='Alma Barcelona GL', polarity=1, prediction=0.0),
 Row(hotel_name='Radisson Blu Edwardian Mercer Street', polarity=1, prediction=1.0),
 Row(hotel_name='The Rockwell', polarity=1, prediction=1.0),
 Row(hotel_name='Sofitel London St James', polarity=1, prediction=1.0),
 Row(hotel_name='NH Amsterdam Caransa', polarity=1, prediction=1.0),
 Row(hotel_name='Hotel Kaiserin Elisabeth', polarity=1, prediction=1.0),
 Row(hotel_name='Crowne Plaza Milan City', polarity=1, prediction=1.0),
 Row(hotel_name='The Marylebone Hotel', polarity=1, prediction=1.0),
 Row(hotel_name='Britannia International Hotel Canary Wharf', polarity=1, prediction=1.0),
 Row(hotel_name='Shaftesbury Metropolis London Hyde Park', polarity=1, prediction=1.0),
 Row(hotel_name='Petit Palace Boqueria Garden', polarity=1, prediction=1.0),
 Row(hotel_name='Mercure Hotel Amsterdam City South', polarity=1, prediction=1.0),
 Row(hotel_name='LHP Hotel Napoleon', polarity=1, prediction=0.0),
 Row(hotel_name='The Hoxton

## Evaluation du Model

In [13]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

my_eval_lr = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='polarity', metricName='areaUnderROC')
my_eval_lr.evaluate(lrPreds)

0.8781162897013849

In [14]:
preds_and_labels= lrPreds.select('prediction','polarity')

y_true = preds_and_labels.select(['polarity']).collect()
y_pred = preds_and_labels.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.90      0.87    101261
           1       0.91      0.86      0.88    122250

    accuracy                           0.88    223511
   macro avg       0.88      0.88      0.88    223511
weighted avg       0.88      0.88      0.88    223511



In [15]:
# Matrice de confusion du RamdomForestClassifier
conf_matrix_lr=confusion_matrix(y_true, y_pred)
conf_matrix_lr = pd.DataFrame(conf_matrix_lr)
conf_matrix_lr

,0,1
0,90958,10303
1,17362,104888


# Modèle avec Grindsearch et CV

In [26]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import numpy as np

In [27]:
lr_best = LogisticRegression(featuresCol = 'features', labelCol = 'polarity', maxIter = 10)

paramGrid = ParamGridBuilder() \
    .addGrid(lr_best.regParam, [0, 0.01]) \
    .build()


crossval_lr = CrossValidator(estimator=lr_best,
                          estimatorParamMaps = paramGrid,
                          evaluator=BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='polarity'),
                          numFolds= 2) 

In [28]:
cvModel_lr = crossval_lr.fit(train)

Py4JJavaError: An error occurred while calling o1405.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 70.0 failed 1 times, most recent failure: Lost task 1.0 in stage 70.0 (TID 223) (dd1593637302 executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.base/java.lang.reflect.Array.newInstance(Array.java:78)
	at java.base/java.io.ObjectInputStream.readArray(ObjectInputStream.java:2066)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1667)
	at java.base/java.io.ObjectInputStream.readArray(ObjectInputStream.java:2102)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1667)
	at java.base/java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2464)
	at java.base/java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2358)
	at java.base/java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2196)
	at java.base/java.io.ObjectInputStream.readObject0(Objec

In [17]:
lrPreds2= cvModel_lr.transform(test)

NameError: name 'cvModel_lr' is not defined